# Vérifier si des publications sont dans la collection HAL d'un laboratoire
Charger les "librairies" (boîtes à outils de code) qui seront nécessaires pour l'opération.
Si nécessaire, les installer d'abord.

In [1]:
import pandas as pd
from tqdm import tqdm
import tkinter as tk
from tkinter import filedialog as fd
from functions import check_df,normalise
from halcollimporter import HalCollImporter
tqdm.pandas()
tk.Tk().withdraw()

''

## Informations données par l'utilisateur

Ecrire dans la fenêtre ouverte par la cellule suivante le code de la collection à vérifier

In [2]:
collection_a_chercher=tk.simpledialog.askstring(prompt="Entrez le code de la collection HAL de votre structure",title='Code de la collection').upper()
fichier=fd.askopenfilename(title='Choisissez le fichier qui contient les publications à vérifier')

## Chargement des données à comparer

Choisir dans la fenêtre ouverte par la cellule suivante le fichier contenant les publications à comparer avec HAL. Ce doit être un fichier Excel.

In [3]:
publis_a_checker=pd.read_excel(fichier)
publis_a_checker.rename({"DOI":'doi',"display_name":"Title","Article Title":"Title","Publication Year":"Year"},axis='columns',inplace=True)
publis_a_checker['Statut']=''
if "Year" in publis_a_checker.columns:
    date_debut=min(publis_a_checker['Year'].tolist())-1
    date_fin=max(publis_a_checker['Year'].tolist())+1
else:
    date_debut=None
    date_fin=None

## Import de la collection HAL

Charger depuis HAL les documents de la collection

In [4]:
coll=HalCollImporter(collection_a_chercher,date_debut,date_fin)
valid=tk.messagebox.askokcancel(title="Importer les publications ?", message=f"{coll.nbdocs} publications trouvées pour {coll.coll_code} sur la période. Les importer ?")
if valid==True:
    coll_df=coll.import_data()

publications trouvées : 2573
terminé : 2573 publications chargées	

## Traitement principal

Comparer les publications du fichier du labo avec celles issues de HAL. C'est l'opération la plus complexe car chaque ligne fait l'objet de deux comparaisons potentielles : DOI et titre.

In [6]:
coll_df['nti']=coll_df['Titres'].apply(lambda x : normalise(x).strip())
check_df(publis_a_checker,coll_df)

100%|██████████| 92/92 [00:14<00:00,  6.22it/s]


## Export du résultat

Réexporter le fichier du labo. Quatre colonnes ont été ajoutées, "Statut", "Titre si trouvé", "Identifiant HAL si trouvé", et "Type de dépôt si trouvé".
- La colonne "Statut" permet de savoir quelles publications sont déjà dans la collection du labo, lesquelles sont dans HAL mais pas dans la collection (probablement mal affiliées), et lesquelles ne sont pas dans HAL.
- La colonne "Titre si trouvé" permet de comparer le titre trouvé dans HAL avec le titre d'origine du document, pour s'assurer qu'il s'agit bien des mêmes. Cette étape est faite par le code mais une vérification manuelle est souvent utile, en particulier lorsque le statut est "Titre inexact trouvé".
- La colonne "Identifiant HAL si trouvé" permet de vérifier directement dans HAL si certaines informations sont manquantes.
- La colonne "Type de dépôt si trouvé" permet, pour les publications présentes dans HAL, de vérifier s'il s'agit seulement de notices ou si le texte intégral est présent.

In [7]:
publis_a_checker.to_excel(fichier.replace(".xlsx","_traite.xlsx"),index=False)